# Installing Necessary Libraries

In [ ]:
!pip install split-folders

# Kaggle Data Import

In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = "xxxx" # username from the json file
os.environ['KAGGLE_KEY'] = "xxxx" # key from the json file
!kaggle datasets download -d apollo2506/eurosat-dataset
!unzip eurosat-dataset.zip -d eurosat

# Necessary Imports

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import numpy as np
import splitfolders

# Dataset Loading And Train Validation Test Split

In [ ]:
input_folder = "eurosat/EuroSAT"
output_folder = "eurosat_split"

# Split the data into training, validation, and test sets
splitfolders.ratio(input_folder, output=output_folder, seed=42, ratio=(0.7, 0.1, 0.2))
data_generator = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2
)

data_path = "eurosat_split"

train_data = data_generator.flow_from_directory(
    data_path + '/train',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    classes=['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture', 'PermanentCrop', 'Residential', 'River', 'SeaLake']
)

validation_data = data_generator.flow_from_directory(
    data_path + '/val',
    target_size=(64, 64),
    batch_size=1,
    class_mode='categorical',
    classes=['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture', 'PermanentCrop', 'Residential', 'River', 'SeaLake']
)

test_data = data_generator.flow_from_directory(
    data_path + '/test',
    target_size=(64, 64),
    batch_size=1,
    class_mode='categorical',
    classes=['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture', 'PermanentCrop', 'Residential', 'River', 'SeaLake']
)

# Model Building

In [ ]:
CNN = keras.Sequential([
        keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Conv2D(64, (3, 3), activation='relu'),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Conv2D(128, (3, 3), activation='relu'),
        keras.layers.Dropout(0.3),
        keras.layers.Conv2D(128, (3, 3), activation='relu'),
        keras.layers.Dropout(0.3),
        keras.layers.Conv2D(64, (3, 3), activation='relu'),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Conv2D(32, (3, 3), activation='relu'),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Flatten(),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(10,activation='softmax')
])
CNN.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Training

In [ ]:
# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Assuming CNN is your compiled model
history = CNN.fit(train_data, validation_data=validation_data, epochs=40, callbacks=[early_stopping])

# Plot Training Validation Accuracy

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Test

In [ ]:
CNN.evaluate(test_data)